In [1]:
import pandas as pd
import numpy as np
import string
import random
import collections
import os
os.environ["DATASET_DIR"] = "/home/rustambaku13/Documents/Warwick/d3-gnn/datasets"

# Tag-Ask-Ubuntu

In [3]:
class TagAskUbuntu:
    def __init__(self):
        n_vertices = pd.read_csv(os.path.join(os.environ["DATASET_DIR"], "tags-ask-ubuntu","tags-ask-ubuntu-nverts.txt"), header=None)[0].values
        simplices = pd.read_csv(os.path.join(os.environ["DATASET_DIR"], "tags-ask-ubuntu","tags-ask-ubuntu-simplices.txt"), header=None)[0].values
        n_labels = pd.read_csv(os.path.join(os.environ["DATASET_DIR"], "tags-ask-ubuntu","tags-ask-ubuntu-node-labels.txt"), header=None, delimiter=" ", usecols=[1])[1].values
        simplex_labels = pd.read_csv(os.path.join(os.environ["DATASET_DIR"], "tags-ask-ubuntu","tags-ask-ubuntu-simplex-labels.txt"), header=None, delimiter=" ")[0].values
        self.q2t = collections.defaultdict(list) # simplex -> [nodes]
        self.t2q = collections.defaultdict(list) # nodes -> [simplex]
        index = 0
        for simplex_idx in range(n_vertices.shape[0]):
            s = str(simplex_labels[simplex_idx])
            for j in simplices[index:index+n_vertices[simplex_idx]]:
                n_label = n_labels[j-1]
                self.t2q[n_label].append(s)
                self.q2t[s].append(n_label)
            index+=n_vertices[simplex_idx]
            
    def create_files(self):
        def create_file(my_dict, destination):
            with open(destination,"w") as f:
                for key, val in my_dict.items():
                    f.write(f'{key},{",".join(val)}\n')
        create_file(self.q2t, os.path.join(os.environ["DATASET_DIR"], "tags-ask-ubuntu","tags-ask-ubuntu[question-tag].txt"))
        create_file(self.t2q, os.path.join(os.environ["DATASET_DIR"], "tags-ask-ubuntu","tags-ask-ubuntu[tag-question].txt"))
        
    def generate_statistics(self):
        num_nodes = len(self.t2q)
        num_hyperedges = len(self.q2t)
        
        

In [4]:
TagAskUbuntu().create_files()

# Reddit-hyperlink

In [5]:
res = pd.read_csv(os.path.join(os.environ["DATASET_DIR"], "RedditHyperlinks","soc-redditHyperlinks-body.tsv"), header=None, delimiter="\t", usecols=[0,1])

## Means

In [6]:
res

,0,1
0,leagueoflegends,teamredditteams
1,theredlion,soccer
2,inlandempire,bikela
3,nfl,cfb
4,playmygame,gamedev
...,...,...
286556,negareddit,debatefascism
286557,mildlynomil,justnomil
286558,mmorpg,blackdesertonline
286559,electricskateboards,askreddit


In [7]:
acc = 0unique i,j in res.iterrows():
    acc += res[:i][res[:i][0] == j[1]].shape[0]
acc = acc / res.shape[0]
acc, res.groupby(0).count()[1].mean()

(121.25355857915069, 10.28464271614686)

In [ ]:
res[0].uniq

# StackOverflow

In [5]:
data = pd.read_csv("../datasets/StackOverflow/sx-stackoverflow.txt", header=None, delimiter=" ")
data.to_csv("../datasets/StackOverflow/sx-stackoverflow.tsv", index=False, header=False)

In [4]:
dataset = dgl.data.RedditDataset()
graph = dataset[0]

In [7]:
graph.ndata["feat"].element_size() * graph.ndata["feat"].nelement()

560979720

In [ ]:

os.environ["DATASET_DIR"] = "/home/rustambaku13/Documents/Warwick/flink-streaming-gnn/jupyter/datasets"
frame = pd.DataFrame({"src":graph.uniEdges()[0], "dest":graph.uniEdges()[1]})
frame.to_csv(os.path.join(os.getenv("DATASET_DIR"), "reddit", "graph.tsv"), header=None, index=None, sep="\t")
np.save(os.path.join(os.getenv("DATASET_DIR"), "reddit", "node_features"), graph.ndata["feat"])
np.save(os.path.join(os.getenv("DATASET_DIR"), "reddit", "node_labels"), graph.ndata["label"])